# Model Monitoring


Beforehand, We need to create an inference service to look at. here is complete code to to it, inpired from datapipeline2

In [1]:
!pip install kserve

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfserving 0.5.1 requires azure-storage-blob<=2.1.0,>=1.3.0, but you have azure-storage-blob 12.9.0 which is incompatible.


In [3]:
## import depandancies
from kubernetes import client , config
from kserve import KServeClient
from kserve import constants
from kserve import utils
from kserve import V1beta1InferenceService
from kserve import V1beta1InferenceServiceSpec
from kserve import V1beta1PredictorSpec
from kserve import V1beta1SKLearnSpec
from kserve import V1beta1XGBoostSpec

In [6]:
# get kserve namespace
namespace = utils.get_default_target_namespace()
### Define the model name, due to technical limitations on service naming size, consider using just a trigram.
name = 'monitor'
### the path to model used to launch the service, this time it's from a shared bucket
bucket='models'
storage_uri=f"s3://{bucket}/frompipeline/xgboost/chicago"

In [5]:
### bind minio credentials to allow kserve getting model
!kubectl apply -f resources/sa.yaml

error: the path "resources/sa.yaml" does not exist


In [7]:
### define inference service from model
chicago_isvc = V1beta1InferenceService(
    api_version="serving.kserve.io/v1beta1",
    kind=constants.KSERVE_KIND,
    metadata=client.V1ObjectMeta(
        name=name,
        namespace=namespace
    ),
    spec=V1beta1InferenceServiceSpec(
        predictor=V1beta1PredictorSpec(
            xgboost=(
                V1beta1XGBoostSpec(
                    storage_uri=storage_uri,
                    protocol_version="v2"
                )
            ),
            service_account_name='kserve-minio-sa',
            image_pull_secrets=[{'name':'registry-secret'}]
        )
    )

)

In [8]:
### Create kserve client
KServe = KServeClient()

In [9]:
### deploy inference service
KServe.create(chicago_isvc)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'creationTimestamp': '2023-01-24T10:14:41Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:imagePullSecrets': {},
       'f:serviceAccountName': {},
       'f:xgboost': {'.': {},
        'f:name': {},
        'f:protocolVersion': {},
        'f:storageUri': {}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2023-01-24T10:14:38Z'}],
  'name': 'monitor',
  'namespace': 'kubeflow-user-guillaume-etevenard',
  'resourceVersion': '7419995771',
  'uid': 'f6bec75a-314a-4700-b202-3909ba5c3081'},
 'spec': {'predictor': {'imagePullSecrets': [{'name': 'registry-secret'}],
   'serviceAccountName': 'kserve-minio-sa',
   'xgboost': {'name': 'kserve-container',
    'protocolVersion': 'v2',
    'resources': {'limits': {'cpu': '1', 'memory': '

### wait until you see this : 

![launched](./images/launched.png)

In [15]:
import requests

### using Kserve api, get the url of the service
isvc_resp = KServe.get(name, namespace=namespace)
isvc_url = isvc_resp['status']['address']['url']

In [16]:
### print the url
isvc_url

'http://monitor.kubeflow-user-guillaume-etevenard.svc.cluster.local/v2/models/monitor/infer'

### Get test data

In [17]:
!pip install pandas

In [18]:
### First create a local folder to handle data
!mkdir chicagodata

In [19]:

# Get the dataset Taxi Trips as CSV
!curl --get 'https://data.cityofchicago.org/resource/wrvz-psew.csv' \
  --data-urlencode '$limit=10000' \
  --data-urlencode '$where=trip_start_timestamp >= "2019-01-01" AND trip_start_timestamp < "2019-02-01"' \
  --data-urlencode '$select=tips,trip_start_timestamp,trip_seconds,trip_miles,pickup_community_area,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_community_area,fare,tolls,extras,trip_total' \
  | tr -d '"' > "./chicagodata/trip.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1087k    0 1087k    0     0  19331      0 --:--:--  0:00:57 --:--:--  241k


In [20]:
import pandas as pd

In [132]:
data = pd.read_csv('chicagodata/trip.csv')

### Create a function that loop on dataset and query service for each line

In [23]:
import requests

In [81]:
data.head()

,tips,trip_start_timestamp,trip_seconds,trip_miles,pickup_community_area,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_community_area,fare,tolls,extras,trip_total
0,0.0,2019-01-31T23:45:00.000,746.0,3.34,6.0,41.944227,-87.655998,16.0,11.75,0.0,0.0,11.75
1,0.0,2019-01-31T23:45:00.000,681.0,3.00,8.0,41.899602,-87.633308,24.0,11.25,0.0,0.0,11.25
2,11.0,2019-01-31T23:45:00.000,2280.0,15.20,56.0,41.792592,-87.769615,22.0,39.00,0.0,5.0,55.00
3,2.0,2019-01-31T23:45:00.000,360.0,1.20,8.0,41.899602,-87.633308,32.0,6.50,0.0,0.0,8.50
4,8.4,2019-01-31T23:45:00.000,1500.0,11.50,76.0,41.980264,-87.913625,4.0,29.75,0.0,4.0,42.15


In [91]:
def stresstestloop(data, maxline, isvc_url):
    # drop target and timestamp columns
    features = data....
    for line in range(maxline):
        ### get line as list
        l = features...
        ### embed into another list like ---> [[x1,x2,x3]]
        list_embedding=...
        ### fill spec template
        inference_input = {
            "inputs": [
                {
                    "name": "input-0",
                    "shape": [1, 10],
                    "datatype": "FP32",
                    "data": list_embedding
                }
            ]
        }
        ### query the model
        response = requests.post(isvc_url, json=inference_input)
#         print(response.text) ## case of debug


### Create a function that asks for batch prediction

In [126]:
def batch_pred(data, isvc_url):
    # drop target
    features = data.drop(columns=["tips","trip_start_timestamp"]).dropna()
    batch = features...
    print(len(batch))
    inference_input = {
        "inputs": [
            {
                "name": "input-0",
                "shape": [..., 10],
                "datatype": "FP32",
                "data": batch
            }
        ]
    }
        ### query the model
    response = requests.post(isvc_url, json=inference_input)
    return response

### Take a look at the monitoring

#### Through Kubeflow

![moniflow](./images/moniflow.png)

#### Through metrics app

go to https://metrics.aiengineer.polytech.sandbox-atos.com/

click browse and choose a frame to look at

![monigraf](./images/monigraf.png)

#### Execute high rate inference

In [129]:
stresstestloop(data,1000,isvc_url)

#### we just did 1000++ query to the model, lets see if the service catches it

![stress](./images/stress.png)

![cpu](./images/cpu.png)



### Execute batch prediction

In [127]:
batch_pred(data,isvc_url)

8258


<Response [200]>

**We just launched 8000++ lines, lets see the response time**

![resptime](./images/resptime.png)

#### Provoque bad response with configuration bias 

In [133]:
def stresstestbiased(data, maxline, isvc_url):
    # drop target and timestamp columns
    features = data...
    for line in range(maxline):
        ### get line as list
        l = features...
        list_embedding =...
        ### you can add bias with inputs metadata, like in putting wrong values for the shape
        inference_input = {
            "inputs": [
                {
                    "name": "input-0",
                    "shape": [..., ...],
                    "datatype": "FP32",
                    "data": list_embedding
                }
            ]
        }
        ### query the model
        response = requests.post(isvc_url, json=inference_input)

In [134]:
stresstestbiased(data,200,isvc_url)

**you should see 4xx class code appear in monitoring**

![400](./images/400.png)